In [2]:
import os
import torch as th

# Print current working directory
print("Current Working Directory:", os.getcwd())

# If necessary, change it to the root directory of your project
os.chdir('../')
print("New Working Directory:", os.getcwd())

from models.core import GPPO
from models.GPPO import train

Current Working Directory: /Users/chiyeong/Documents/projects/winter-study-reinforcement/RL-Scheduler/test
New Working Directory: /Users/chiyeong/Documents/projects/winter-study-reinforcement/RL-Scheduler


In [3]:
dir_path = os.path.join(os.getcwd(), 'instances')

instance_configs = {
    'train': [],
    'val': [],
    'test': []
}

for i in range(1, 11):
    config = {
        'type': 'standard',
        'path': os.path.join(dir_path, f'standard/ta{str(i).zfill(2)}'),
        'repeat': [1] * 15
    }
    if i < 7:
        instance_configs['train'].append(config)
    elif i < 9:
        instance_configs['val'].append(config)
    else:
        instance_configs['test'].append(config)

In [4]:

device = 'cpu'
if th.cuda.is_available():
    device = 'cuda'
elif th.backends.mps.is_available():
    device = 'mps'
print(f'Using {device} device')


Using mps device


In [5]:

gppo = GPPO(device=device)
best_model = train(actor_critic=gppo,
                    instance_configs=instance_configs,
                    steps_per_epoch=10,
                    epochs=10,)

# Save best model
th.save(best_model, 'best_model.pth')


Logging data to /tmp/experiments/1721357412/progress.txt

Number of parameters: 	 pi: 12481, 	 v: 8385

New best model found!
---------------------------------------
|             Epoch |               0 |
|      AverageVVals |          0.0708 |
|          StdVVals |        8.54e-05 |
|          MaxVVals |          0.0709 |
|          MinVVals |          0.0706 |
| TotalEnvInteracts |              10 |
|            LossPi |        0.000111 |
|             LossV |        3.61e+04 |
|       DeltaLossPi |         -0.0583 |
|        DeltaLossV |       -2.88e+03 |
|           Entropy |            2.71 |
|                KL |        -0.00366 |
|          ClipFrac |             0.2 |
|          StopIter |              79 |
|              Time |            16.9 |
|      ValMeanEpRet |       -1.68e+03 |
|      ValMeanEpLen |             225 |
---------------------------------------
Early stopping at step 0 due to reaching max kl.
---------------------------------------
|             Epoch |    

In [6]:
from scheduler_env.GraphJSSPEnv import GraphJSSPEnv

# Load best model
model = GPPO()
model.load_state_dict(th.load('best_model.pth'))
model.eval()

# Test
test_envs = [GraphJSSPEnv(config) for config in instance_configs['test']]
for env in test_envs:
    step = 0
    obs, _ = env.reset()
    done = False
    total_reward = 0

    while not done:
        step += 1

        action = model.act(obs['graph'].data, obs['candidate_op_indices'])
        obs, reward, terminated, truncated, info = env.step(action)
        done = terminated or truncated
        total_reward += reward

        if done:
            print("Goal reached!")
            print(step, info, total_reward)
            obs['graph'].visualize_graph()
            env.render()

TypeError: 'module' object is not callable